In [1]:
import os
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
import datetime as dt
import pandas_ta as ta
import copy
import random
from pymongo import MongoClient
import sys

import warnings
import time
from sqlalchemy import create_engine, text

import feedparser
import requests
from bs4 import BeautifulSoup
import google.generativeai as genai
import os
import re
from bs4 import Tag

warnings.filterwarnings("ignore")
warnings.simplefilter('ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'import'))
from import_env import load_env

mongo_client = MongoClient(load_env("MONGO_URI"))
stock_db = mongo_client["stock_db"]

vsuccess_uri = load_env("VSUCCESS_URI")
if not vsuccess_uri:
	raise ValueError("Environment variable 'VSUCCESS_URI' is not set.")
vsuccess_engine = create_engine(vsuccess_uri)

In [2]:
def get_article_content(url):

    # Set up headers for the request
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
    
    try:
        # Get webpage content
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        # Parse HTML
        soup = BeautifulSoup(response.content, "html.parser")
        # Find article content
        content_div = soup.find("div", {"itemprop": "articleBody", "id": "vst_detail"})
        
        # Ensure content_div is a Tag object before proceeding
        if not isinstance(content_div, Tag):
            return ""
        
        # Extract text from paragraphs
        article_content = ""
        paragraphs = content_div.find_all("p")
        
        for p in paragraphs:
            # Skip author and publishing info
            if p.get("class") in [["pAuthor"], ["pPublishTimeSource", "right"]]:
                continue
            
            text = p.get_text(strip=True)
            if text:
                article_content += f"{text}\n"
        
        return article_content.strip()
        
    except Exception:
        return ""
    
def get_summary(model, content, news_type):
    if news_type == 'doanh_nghiep':
        promt = f"""
            Hãy tóm tắt nội dung của bài báo được cung cấp dưới đây:
            {content}

            Yêu cầu cho bản tóm tắt:

            1. Phân tích nội dung để xác định mã cổ phiếu chính mà bài báo tập trung đề cập hoặc phân tích.
            2. Bản tóm tắt phải bắt đầu bằng chính mã cổ phiếu đó, theo định dạng: [MÃ CỔ PHIẾU]: Nội dung tóm tắt... (Ví dụ: HPG: Công ty vừa công bố kế hoạch mở rộng sản xuất thép...).
            3. Nội dung tóm tắt theo sau là một đoạn văn liền mạch, khoảng 30 từ, trình bày các thông tin cốt lõi liên quan đến mã cổ phiếu đó từ bài báo.
            4. Nếu bài báo không đề cập đến mã cổ phiếu cụ thể nào một cách rõ ràng, hoặc không có mã nào là trọng tâm thì trình bày nội dung tóm tắt như bình thường, không cần bắt đầu bằng mã cổ phiếu.
            5. Phần nội dung bắt đầu trực tiếp bằng thông tin chính của bài báo, không sử dụng các cụm từ giới thiệu như "Bài báo này nói về...", "Theo bài báo thì...", "Nội dung của bài viết là...". Hãy hành văn như đang cung cấp tin tức.
            6. Đặc biệt quan trọng: Phải bao gồm đầy đủ và chính xác tất cả các số liệu cụ thể đã được đề cập trong bài báo gốc. Vui lòng viết bằng tiếng Việt.
        """
    else:
        promt = f"""
            Hãy tóm tắt nội dung của bài báo được cung cấp dưới đây:
            {content}

            Yêu cầu cho bản tóm tắt:

            1. Trình bày thành một đoạn văn duy nhất, liền mạch.
            2. Bắt đầu trực tiếp bằng thông tin chính của bài báo, không sử dụng các cụm từ giới thiệu như "Bài báo này nói về...", "Theo bài báo thì...", "Nội dung của bài viết là...". Hãy hành văn như đang cung cấp tin tức.
            3. Tổng độ dài toàn bộ các mẩu tin khoảng 30 từ.
            4. Đặc biệt quan trọng: Phải bao gồm đầy đủ và chính xác tất cả các số liệu cụ thể đã được đề cập trong bài báo gốc. Vui lòng viết bằng tiếng Việt.
        """

    response = model.generate_content(promt)
    return response.text

def split_news_content(content):
    points = content.split('- ')
    cleaned_points = []
    for point in points:
        point = point.strip()
        if point:
            cleaned_points.append(point)
    
    return cleaned_points

In [3]:
rss_url_dict = {
    'vi_mo': {
        'vi_mo': 'https://vietstock.vn/761/kinh-te/vi-mo.rss',
        'kinh_te_dau_tu': 'https://vietstock.vn/768/kinh-te/kinh-te-dau-tu.rss',
    },
    'chinh_sach': {
        'chinh_sach': 'https://vietstock.vn/143/chung-khoan/chinh-sach.rss',
    },
    'quoc_te': {
        'tai_chinh_quoc_te': 'https://vietstock.vn/772/the-gioi/tai-chinh-quoc-te.rss',
        'chung_khoan_the_gioi': 'https://vietstock.vn/773/the-gioi/chung-khoan-the-gioi.rss',
    },
    'doanh_nghiep': {
        'hoat_dong_kinh_doanh': 'https://vietstock.vn/737/doanh-nghiep/hoat-dong-kinh-doanh.rss',
    }
}

genai.configure(api_key=load_env("GEMINI_API")) # type: ignore
# gemini = genai.GenerativeModel('gemini-2.5-flash-preview-04-17-thinking') # type: ignore
gemini = genai.GenerativeModel('gemini-2.0-flash') # type: ignore

full_news_df_dict = {}
for news_type, url_dict in rss_url_dict.items():
    temp_news_list = []
    max_entries = 1 if news_type in ['doanh_nghiep', 'chinh_sach'] else 1
    
    for url in url_dict.values():
        feed = feedparser.parse(url)
        for entry in feed.entries[:max_entries]:
            content = get_article_content(entry['id'])
            
            # Lấy thời gian đăng bài từ RSS feed
            published_time = ""
            if hasattr(entry, 'published') and entry.published:
                published_time = entry.published
            
            temp_news_list.append({
                'title': entry['title'], 
                # 'content': get_summary(gemini, content, news_type),
                'content': content,
                'published_time': published_time
            })
            # time.sleep(2)
    full_news_df_dict[news_type] = temp_news_list

In [ ]:
filtered_news_dict = {}
for news_type in full_news_df_dict.keys():
    filtered_news_list = []
    for item in full_news_df_dict[news_type]:
        content = item['content']
        if 'bài báo' in content.lower():
            continue
        # Đếm số từ
        word_count = len(content.split())
        # Kiểm tra số từ trong khoảng 40-60
        if 40 <= word_count <= 70:
            filtered_news_list.append(item)
        else:
            print(f"Đã loại {news_type} với số từ: {word_count}")


    filtered_news_dict[news_type] = filtered_news_list

In [ ]:
data_list = []
for news_type, news_list in filtered_news_dict.items():
    # Create a temporary DataFrame for sorting
    temp_df = pd.DataFrame(news_list)
    if not temp_df.empty and 'published_time' in temp_df.columns:
        # Convert 'published_time' to datetime objects using the correct format
        # RFC 822 format (common in RSS feeds) uses abbreviated month names (%b)
        temp_df['published_time'] = pd.to_datetime(
            temp_df['published_time'],
            format="%a, %d %b %Y %H:%M:%S %z", 
            errors='coerce'  # If a date can't be parsed with this format, it becomes NaT
        )
        
        # Drop rows where 'published_time' could not be converted (is NaT)
        temp_df.dropna(subset=['published_time'], inplace=True)
        
        if not temp_df.empty: # Proceed only if temp_df has valid, parsed dates
            temp_df = temp_df.sort_values(by='published_time', ascending=False)
            
            for index, row in enumerate(temp_df.itertuples(), 1):
                data_list.append({
                    'news_type': news_type,
                    'news_number': index,
                    'title': row.title,
                    'content': row.content,
                    'published_time': row.published_time # This is now a timezone-aware datetime object
                })

daily_news_df = pd.DataFrame(data_list)

if not daily_news_df.empty and 'published_time' in daily_news_df.columns and pd.api.types.is_datetime64_any_dtype(daily_news_df['published_time']):
    daily_news_df['published_time'] = daily_news_df['published_time'].dt.tz_convert('Asia/Ho_Chi_Minh').dt.tz_localize(None)
elif not daily_news_df.empty and 'published_time' in daily_news_df.columns: # Handle cases where column might exist but not as datetime64 (e.g. all NaT)
    # Attempt conversion if not already datetime, though previous steps should ensure it
    daily_news_df['published_time'] = pd.to_datetime(daily_news_df['published_time'], errors='coerce')
    daily_news_df.dropna(subset=['published_time'], inplace=True) # drop if conversion failed
    if not daily_news_df.empty and pd.api.types.is_datetime64_any_dtype(daily_news_df['published_time']):
        daily_news_df['published_time'] = daily_news_df['published_time'].dt.tz_convert('Asia/Ho_Chi_Minh').dt.tz_localize(None)

daily_news_df

In [ ]:
daily_news_df.to_sql('daily_news_df', vsuccess_engine, if_exists='replace', index=False)